In [ ]:
import sys
sys.path.append('..')
from config_figures import *

In [ ]:
df = pd.read_csv(nmds_evopca_covariates_file).drop(columns=['system:index','.geo'])
df

# PCA environmental variables

In [ ]:
# compute PCA on environmental covariates
df_covs = df[[
    'CHELSA_bio12_1981_2010_V2_1', 'CHELSA_bio15_1981_2010_V2_1', 'CHELSA_bio1_1981_2010_V2_1', 
    'CHELSA_bio4_1981_2010_V2_1', 'CHELSA_gsl_1981_2010_V2_1', 'CHELSA_npp_1981_2010_V2_1', 
    'SG_Coarse_fragments_005cm', 'SG_Silt_Content_005cm', 'SG_Soil_pH_H2O_005cm'
]]
df_st = StandardScaler().fit_transform(df_covs)
pcamodel = PCA(n_components=2)
pca = pd.DataFrame(pcamodel.fit_transform(df_st)).rename(columns = {0:'pc1', 1:'pc2'})
scaled_pca = pca * (1.0 / (pca.max(axis=0) - pca.min(axis=0)))
scaled_pca

In [ ]:
# compute variance explained by PCs
pca_variance_explained = pd.Series(pcamodel.explained_variance_ratio_, index=['pc1','pc2'])
pca_variance_explained

In [ ]:
# compute loadings of environmental covariates in PC space
pca_loadings = pd.DataFrame(pcamodel.components_.T, index = df_covs.columns, columns=['pc1','pc2'])
pca_loadings['var_name'] = ['Annual P', 'P season', 'MAT', 'T season', 'GSL', 'NPP', 'CF', 'Silt', 'pH']
pca_loadings

In [ ]:
# merge ordinations with PCA dataframe
df2 = scaled_pca.join(df[['MDS1','MDS2','MDS3','Axis1','Axis2','Axis3','x','y','area']])
df2

# Map 10%-90% quantiles to 0-1 for RGB color mapping

In [ ]:
# scale ordinations (10-90% quantiles mapped to 0-1) for colors
def scale_to_0_1(vec):
    q10 = vec.quantile(0.1)
    q90 = vec.quantile(0.9)
    return vec.apply(lambda x: min(max((x-q10)/(q90-q10), 0), 1))

df2[['MDS1_blue','MDS2_green','MDS3_red']] = df2[['MDS1','MDS2','MDS3']].apply(scale_to_0_1)
nmds_colors = np.array(df2[['MDS3_red','MDS2_green','MDS1_blue']])
df2[['Axis1_blue','Axis2_green','Axis3_red']] = df2[['Axis1','Axis2','Axis3']].apply(scale_to_0_1)
evopca_colors = np.array(df2[['Axis3_red','Axis2_green', 'Axis1_blue']])
df2

# Plot NMDS and evoPCA in envioronmental PC-space

In [ ]:
# figure with ordinations mapped into PC space 
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,8))

ax1.scatter(x=df2['pc1'], y=df2['pc2'], c=nmds_colors, s=(df2['area']*1e-10), alpha=0.5)
ax1.set_title('Taxonomic ordination (NMDS)', fontsize=20)
ax2.scatter(x=df2['pc1'], y=df2['pc2'], c=evopca_colors, s=(df2['area']*1e-10), alpha=0.5)
ax2.set_title('Phylogenetic ordination (evoPCA)', fontsize=20)

for ax in [ax1, ax2]:
    ax.set_xlabel("Environmental PC1 (" + str(pca_variance_explained.pc1.round(3) * 100) + "%)", fontsize=12)
    ax.set_ylabel( "Environmental PC2 (" + str(pca_variance_explained.pc2.round(3) * 100) + "%)", fontsize=12)
    for r in pca_loadings.iterrows():
        ax.arrow(0, 0, r[1]['pc1'], r[1]['pc2'], color = 'k')
        text_x = r[1]['pc1'] + 0.04 if r[1]['pc1'] >= 0 else r[1]['pc1'] - 0.04
        text_y = r[1]['pc2'] + 0.04 if r[1]['pc2'] >= 0 else r[1]['pc2'] - 0.04
        ax.text(text_x, text_y, r[1]['var_name'], color = 'k', ha = 'center', va = 'center', fontsize=12)
    ax.set(xlim = (-0.7, 0.7), ylim = (-0.7, 0.7), xticklabels=[], xticks=[], yticklabels=[], yticks=[])
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

plt.tight_layout()
plt.savefig(figuredir + 'nmds_evopca_pca.png')

# Plot NMDS and evoPCA in 2d + distributions in 1d and 2d

In [ ]:
df2[['MDS1','MDS2','MDS3','Axis1','Axis2','Axis3']].quantile([0,0.01,0.05,0.95,0.99,1])

In [ ]:
nbins = 50

fig = plt.figure(layout='constrained', figsize=(12,8))
subfigs = fig.subfigures(2, 1, hspace=0.05)

subfigs[0].suptitle('Taxonomic ordination (NMDS)')
subfigs_top = subfigs[0].subfigures(1, 2, wspace=0.05, width_ratios=[3,1])
axs_top_left = subfigs_top[0].subplots(2,3)

lim1 = (-0.8,0.8)
lim2 = (-0.8,0.8)
lim3 = (-0.7,0.7)

ax1, ax2, ax3 = axs_top_left[0]
ax1.scatter(df2['MDS1'], df2['MDS2'], s=(df2['area']*1e-10), alpha=0.5, c=nmds_colors)
ax1.set(xlabel =None, ylabel='NMDS2', xlim=lim1, ylim=lim2, xticklabels=[])
ax2.scatter(df2['MDS2'], df2['MDS3'], s=(df2['area']*1e-10), alpha=0.5, c=nmds_colors)
ax2.set(xlabel =None, ylabel='NMDS3', xlim=lim2, ylim=lim3, xticklabels=[])
ax3.scatter(df2['MDS3'], df2['MDS1'], s=(df2['area']*1e-10), alpha=0.5, c=nmds_colors)
ax3.set(xlabel=None, ylabel='NMDS1', xlim=lim3, ylim=lim1, xticklabels=[])

ax1, ax2, ax3 = axs_top_left[1]
sns.histplot(df2, x='MDS1', y='MDS2', weights='area', bins=nbins, binrange=(lim1, lim2), ax=ax1)
ax1.set(xlabel='NMDS1', ylabel='NMDS2', xlim=lim1, ylim=lim2)
sns.histplot(df2, x='MDS2', y='MDS3', weights='area', bins=nbins, binrange=(lim2, lim3), ax=ax2)
ax2.set(xlabel='NMDS2', ylabel='NMDS3', xlim=lim2, ylim=lim3)
sns.histplot(df2, x='MDS3', y='MDS1', weights='area', bins=nbins, binrange=(lim3, lim1), ax=ax3)
ax3.set(xlabel='NMDS3', ylabel='NMDS1', xlim=lim3, ylim=lim1)

ax1, ax2, ax3 = subfigs_top[1].subplots(3,1)
sns.histplot(df2, x='MDS1', weights='area', stat='density', bins=nbins, binrange=lim1, color='blue', ax=ax1)
ax1.set(xlabel = 'NMDS1', ylabel=None, yticklabels=[], yticks=[], xlim=lim1)
sns.histplot(df2, x='MDS2', weights='area', stat='density', bins=nbins, binrange=lim2, color='green', ax=ax2)
ax2.set(xlabel ='NMDS2', ylabel='Histograms (density)', yticklabels=[], yticks=[], xlim=lim2)
sns.histplot(df2, x='MDS3', weights='area', stat='density', bins=nbins, binrange=lim3, color='red', ax=ax3)
ax3.set(xlabel ='NMDS3', ylabel=None, yticklabels=[], yticks=[], xlim=lim3)

subfigs[1].suptitle('Phylogenetic ordination (evoPCA)')
subfigs_bottom = subfigs[1].subfigures(1, 2, wspace=0.05, width_ratios=[3,1])
axs_bottom_left = subfigs_bottom[0].subplots(2,3)
lim1 = (-0.8,0.3)
lim2 = (-0.4,0.3)
lim3 = (-0.3,0.3)

ax1, ax2, ax3 = axs_bottom_left[0]
ax1.scatter(df2['Axis1'], df2['Axis2'], s=(df2['area']*1e-10), alpha=0.5, c=evopca_colors)
ax1.set(xlabel =None, ylabel='evoPCA2', xticklabels=[], xlim=lim1, ylim=lim2)
ax2.scatter(df2['Axis2'], df2['Axis3'], s=(df2['area']*1e-10), alpha=0.5, c=evopca_colors)
ax2.set(xlabel =None, ylabel='evoPCA3', xticklabels=[], xlim=lim2, ylim=lim3)
ax3.scatter(df2['Axis3'], df2['Axis1'], s=(df2['area']*1e-10), alpha=0.5, c=evopca_colors)
ax3.set(xlabel=None, ylabel='evoPCA1', xticklabels=[], xlim=lim3, ylim=lim1)

ax1,ax2, ax3 = axs_bottom_left[1]
sns.histplot(df2, x='Axis1', y='Axis2', weights='area', bins=nbins, binrange=(lim1,lim2), ax=ax1)
ax1.set(xlabel='evoPCA1', ylabel='evoPCA2', xlim=lim1, ylim=lim2)
sns.histplot(df2, x='Axis2', y='Axis3', weights='area', bins=nbins, binrange=(lim2,lim3), ax=ax2)
ax2.set(xlabel='evoPCA2', ylabel='evoPCA3', xlim=lim2, ylim=lim3)
sns.histplot(df2, x='Axis3', y='Axis1', weights='area', bins=nbins, binrange=(lim3,lim1), ax=ax3)
ax3.set(xlabel='evoPCA3', ylabel='evoPCA1', xlim=lim3, ylim=lim1)

ax1, ax2, ax3 = subfigs_bottom[1].subplots(3, 1)
sns.histplot(df2, x='Axis1', weights='area', stat='density', bins=nbins, binrange=lim1, color='blue', ax=ax1)
ax1.set(xlabel = 'evoPCA1', ylabel=None, yticklabels=[], yticks=[], xlim=lim1)
sns.histplot(df2, x='Axis2', weights='area', stat='density', bins=nbins, binrange=lim2, color='green', ax=ax2)
ax2.set(xlabel ='evoPCA2', ylabel='Histograms (density)', yticklabels=[], yticks=[], xlim=lim2)
sns.histplot(df2, x='Axis3', weights='area', stat='density', bins=nbins, binrange=lim3, color='red', ax=ax3)
ax3.set(xlabel ='evoPCA3', ylabel=None, yticklabels=[], yticks=[], xlim=lim3)

plt.savefig(figuredir + 'nmds_evopca_supp.png')

# Clustering analysis

In [ ]:
# remove outliers from ordinations
df_ordinations = df2[['MDS1','MDS2','MDS3','Axis1','Axis2','Axis3']]
q1 = df_ordinations.quantile(0.25)
q3 = df_ordinations.quantile(0.75)
iqr = q3 - q1
df_nooutliers = df2[df_ordinations.apply(lambda x: (x >= q1[x.name] - 1.5*iqr[x.name]) & (x <= q3[x.name] + 1.5*iqr[x.name])).all(axis=1)]
df_nooutliers

In [ ]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_nooutliers[['MDS1','MDS2','MDS3','Axis1','Axis2','Axis3']]), columns = ['MDS1','MDS2','MDS3','Axis1','Axis2','Axis3'])
df_scaled = pd.concat([df_scaled, df_nooutliers[['x','y','area','MDS1_red','MDS2_green','MDS3_blue','Axis1_green','Axis2_red','Axis3_blue']].reset_index()], axis=1)
df_scaled

In [ ]:
nmds = df_scaled[['MDS1','MDS2','MDS3']]
nmds_scores = []
for k in range(2,50):
    labels = KMeans(n_clusters=k).fit_predict(nmds)
    score = silhouette_score(nmds, labels)
    nmds_scores.append(score)

In [ ]:
evopca = df_scaled[['Axis1','Axis2','Axis3']]
evocpca_scores = []
for k in range(2,50):
    labels = KMeans(n_clusters=k).fit_predict(evopca)
    score = silhouette_score(evopca, labels)
    evocpca_scores.append(score)

# Cluster analysis plot

In [ ]:
max_k = 20
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(4,8))

sns.lineplot(x=range(2,max_k), y=nmds_scores[:max_k-2], ax=ax1)
ax1.set(ylabel='Silhouette score', ylim=(0.25,0.55), title='Taxonomic ordination (NMDS)')
ax1.axvline(np.argmax(nmds_scores)+2, color='red', linestyle="--", linewidth=1)
ax1.text(np.argmax(nmds_scores)+3, max(nmds_scores), "k=" + str(np.argmax(nmds_scores)+2), fontsize=12, color='red')

sns.lineplot(x=range(2,max_k), y=evocpca_scores[:max_k-2], ax=ax2)
ax2.set(xlabel='Number of clusters', ylabel='Silhouette score', title='Phylogenetic ordination (evoPCA)')
ax2.axvline(np.argmax(evocpca_scores)+2, color='red', linestyle="--", linewidth=1)
ax2.text(np.argmax(evocpca_scores)+3, max(evocpca_scores), "k=" + str(np.argmax(evocpca_scores)+2), fontsize=12, color='red')

plt.savefig(figuredir + 'nmds_evopca_supp_cluster.png')

# Save csv of best clustering solution

In [ ]:
df_scaled['nmds_cluster'] = KMeans(n_clusters=2).fit_predict(nmds)
df_scaled['evopca_cluster'] = KMeans(n_clusters=5).fit_predict(evopca)
df_scaled[['x','y','nmds_cluster','evopca_cluster']].to_csv(datadir + 'ordinations_equal_area_cluster.csv', index=False)